# Import Modules

In [1]:
import duckdb
import pandas as pd
%load_ext sql

Tip: You may define configurations in /Users/kul/Dev/featlib/pyproject.toml or /Users/kul/.jupysql/config.

Did not find user configurations in /Users/kul/Dev/featlib/pyproject.toml.

# Connect to Local Database

In [2]:

# Create connection to local duckdb database
con = duckdb.connect('../db/duck.db')
%sql con --alias duckdb

# Create Features

In [ ]:
query ="""
CREATE OR REPLACE TEMP TABLE tmp_feat_response AS (
    WITH price AS (
        SELECT
            *,
        FROM pricing
        order by act_symbol, date
    ),
    resp AS (
        SELECT
            date_trunc('day', date) AS date, 
            act_symbol,
            (LEAD(close, 5) OVER (PARTITION BY act_symbol ORDER BY date ASC) - close) / close AS fwd_return_1w,
            (LEAD(close, 21) OVER (PARTITION BY act_symbol ORDER BY date ASC) - close) / close AS fwd_return_1m,
            (LEAD(close, 63) OVER (PARTITION BY act_symbol ORDER BY date ASC) - close) / close AS fwd_return_3m,
            (LEAD(close, 126) OVER (PARTITION BY act_symbol ORDER BY date ASC) - close) / close AS fwd_return_6m,
        FROM price
    )
    select *
    from resp
);
"""

con.sql(query)

Unpivot momentum features (horiz -> vert) and insert into table

In [21]:
con.sql("""
CREATE OR REPLACE TABLE features_v_res AS (
    with cte as (
        UNPIVOT tmp_feat_response
        ON COLUMNS(* EXCLUDE (date, act_symbol))
        INTO
            NAME id
            VALUE value
    )
    select 
        date_trunc('day',date) as date, 
        act_symbol as symbol,
        id,
        value
    from cte
)
"""
)

Merge into features table

In [ ]:
con.sql("""
INSERT INTO features_raw
    SELECT * from features_v_res
""")


Check merge executed successfully

In [43]:
%%sql
select count(*) from features_v_res

Running query in 'duckdb'

count_star()
76647608


In [44]:
%%sql
select count(*) from features where id in ('fwd_return_1w', 'fwd_return_1m', 'fwd_return_3m', 'fwd_return_6m')

Running query in 'duckdb'

count_star()
76647608


# Close Database Connection

In [45]:
con.close()